### Import TensorFlow and other libraries

In [80]:
import os
import time
import numpy as np
import tensorflow as tf
from warnings import filterwarnings
filterwarnings("ignore")

# Read the data

In [82]:
text = open('shakespeare.txt', 'rb').read().decode(encoding='utf-8')

In [83]:
vocab = ['[UNK]'] + sorted(set(text)) # Create the vocabulary and add a token UNK that represents Unknown token.

# Process the text

- **Prepare the dependencies**

In [84]:
char_to_id = {c: i for i, c in enumerate(vocab)}
id_to_char = {i: c for i, c in enumerate(vocab)}

def id_from_char(chars):
    return [char_to_id[c] for c in chars]

def char_from_id(ids):
    return [id_to_char[i] for i in ids]

In [85]:
def ids_from_chars(data):
    return np.array([id_from_char(chars) for chars in data])

def chars_from_ids(data):
    return np.array([char_from_id(ids) for ids in data])

In [86]:
def slice_sentences(sentences):
    return [list(sentence) for sentence in sentences]

def construct_sentences(sentences):
    return ["".join(sentence) for sentence in sentences]

### Create training examples and targets

In [87]:
sliced_text = list(text)
all_ids = ids_from_chars(sliced_text)

In [88]:
seq_length = 100
sequence_length = seq_length + 1
sequences_num = len(all_ids) // sequence_length
num_chars_to_take = sequence_length * sequences_num

In [89]:
ids_dataset = np.array(all_ids[:num_chars_to_take])

In [90]:
sequences = ids_dataset.reshape((sequences_num, sequence_length))

### Split data into X & y

In [91]:
def split(sequences):
  X, y = [], []
  for sequence in sequences:
    X.append(sequence[:-1])
    y.append(sequence[1:])
  return np.array(X), np.array(y)

In [92]:
X, y = split(sequences)

## Build The Model

In [93]:
EPOCHS = 20
BATCH_SIZE = 64
rnn_units = 1024
embedding_dim = 256
vocab_size = len(vocab)

In [94]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [95]:
model = MyModel(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)

In [96]:
# model.build(input_shape=(None, 100))
# model.summary()

In [97]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [98]:
# history = model.fit(X, y, epochs=EPOCHS)

# Save the model

In [99]:
# model.save_weights('my_weights.h5')

# Load the model

In [100]:
model = MyModel(vocab_size=vocab_size, embedding_dim=embedding_dim, rnn_units=rnn_units)
model.build(input_shape=(None, 100))
model.load_weights('new_weights.h5')

# Generate Text

In [101]:
def softmax(logits):
  return np.exp(logits) / np.exp(logits).sum()

def generate_one_step(inputs, states=None):
  chars = slice_sentences(inputs)
  ids = ids_from_chars(chars)

  pred_mask = np.zeros((len(vocab)))
  pred_mask[0] = -float('inf')

  pred, states = model(inputs=ids, states=states, return_state=True)
  pred = pred.numpy()
  pred = pred[:, -1, :]
  pred = pred + pred_mask

  prob = softmax(pred).flatten()
  predicted_ids = np.random.choice(range(len(prob)), 1, p=prob)
  predicted_char = char_from_id(predicted_ids)

  return predicted_char, states

In [102]:
states = None
next_char = np.array(["Romeo:"])
result = [next_char[0]]

for n in range(1000):
  next_char, states = generate_one_step(next_char, states=states)
  result.append(next_char[0])

result = construct_sentences([result])
result = result[0]

In [103]:
print(result)

Romeo: canst thou utter 'Gains;
But now mischance commund thee here and tell me
As I intend against a boll word out.
My queen and Henry living uncharms.
Now, good ship warlike words will not yield.

MENENIUS:
What 'twas the gods that we met, my lord?

AUFIDIUS:
All this is lost, or else re'e blood devost
In our about your company.

POLIXENES:
Sir, indeed.

LEONTES:
I think I thank you, sir. 'TRAnk me, you're more.

AUTOLYCUS:
No, holy fath, my humbly I must content;
Come you to us, I'll vent to woo'r his happiness;
'Twas yet a shower so loush'd to hear more
joints worse pricts. Thou art not chose:
Therefore, this is strange: let it be armied,
Which sometill up and doubt, with't have you nield
More greying drops imperting and put to death,
Having but adoined it?
For George and my lord, I hold it now;
And, for I remember you with him!
Travel your learness from the King of Hences,
Who settled his court in Esseluty,
And, then by rights with age, since is no grief it so:
Lest who is left us